In [ ]:
from transformers import BertTokenizer
import pandas as pd
from tensorflow.keras import layers, models
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report

In [ ]:
def split_data(df):
    from sklearn.model_selection import train_test_split
    x = df['text']
    y = df['label']
    return train_test_split(x,y,test_size = 0.2,random_state=42)

def token_data(texts, tokenizer, max_length=512):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='tf'
    )
    return encodings

def performance(y_true,y_pred):
# y_pred son los valores predecidos mientras que y_true son los valores reales
    accuracy = accuracy_score(y_true,y_pred)
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    print(classification_report(y_true, y_pred))
    return{
        'accuracy':accuracy,
        'recall':recall,
        'f1':f1,
    }


def create_model(inputs, num,  num_filters=128, kernel_size=5, pool_size=2):
    model = models.Sequential()
    model.add(layers.Conv1D(num_filters, kernel_size, activation='gelu', input_shape=inputs))
    model.add(layers.MaxPooling1D(pool_size=pool_size))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(num_filters * 2, kernel_size, activation='gelu'))
    model.add(layers.MaxPooling1D(pool_size=pool_size))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation = 'gelu'))
    model.add(layers.Dense(num,activation = 'softmax'))
    return model

In [ ]:
def categorize(text):
    categories = {
        "malware": "Malware",
        "phishing": "Phishing",
        "ransomware": "Ransomware",
        "trojan": "Trojan",
        "worm": "Worm",
        "spyware": "Spyware",
        "ddos": "DDoS",
        "distributed denial of service": "DDoS",
        "zero day": "Zero Days",
        "data breach": "Data Breach",
        "social engineering": "Social Engineering"
    }
    lower_text = text.lower()
    for keyword, category in categories.items():
        if keyword in lower_text:
            return category
    return "Other"

def labeler(df):
    df["label"] = df["text"].apply(categorize)
    return df

In [ ]:
def main():
    # LOAD
    #df = clean_data.limpieza(ldata.ldata1(), ldata.ldata2(), ldata.ldata3())
    df = pd.read_csv('/clean_data.csv')
    print("data loaded")
    #LABEL
    df = labeler(df)
    print("data labeled")
    # SPLIT
    x_train, x_test, y_train, y_test = split_data(df)
    print("data splitted")
    # TOKENIZE
    train_texts = x_train.tolist()
    val_texts = x_test.tolist()
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    train_encodings = token_data(train_texts, tokenizer)
    val_encodings = token_data(val_texts,tokenizer)
    print("data tokenized")
    # Actually training the model
    y_train = pd.get_dummies(y_train).values
    y_test = pd.get_dummies(y_test).values
    inputs = (train_encodings['input_ids'].shape[1], 1)
    num = y_train.shape[1]
    print("creando modelo")
    model = create_model(inputs, num)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_encodings['input_ids'],y_train,epochs=10, batch_size=32, validation_data=(val_encodings['input_ids'],y_test))
#    y_pred =[] # reemplazar
#    y_true = [] # reemplazar
#    metrics = performance(y_true, y_pred)
#    print(metrics)


if __name__ == "__main__":
    main()

data loaded
creando modelo
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


97/97 ━━━━━━━━━━━━━━━━━━━━ 15s 87ms/step - accuracy: 0.4700 - loss: 1.7194 - val_accuracy: 0.4890 - val_loss: 1.8989
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5417 - loss: 1.3191 - val_accuracy: 0.5136 - val_loss: 1.4630
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5796 - loss: 1.2156 - val_accuracy: 0.4021 - val_loss: 1.6372
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5886 - loss: 1.1702 - val_accuracy: 0.5149 - val_loss: 1.4629
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5862 - loss: 1.1715 - val_accuracy: 0.3930 - val_loss: 1.5434
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6050 - loss: 1.1519 - val_accuracy: 0.3632 - val_loss: 1.6699
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6302 - loss: 1.0901 - val_accuracy: 0.5071 - val_loss: 1.3687
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6094 - loss: 1.1151 - val_accuracy: 0.5486 - val_loss: 1.3132
E

SIGUIENTE INTENTOOOOO

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
import pandas as pd

# RECORRER TEXTO PARA CREAR CATEGORIAS 
def categorize(text):
    # DICCIONARIO DE CATEGORIAS
    categories = {
        "malware": "Malware",
        "phishing": "Phishing",
        "ransomware": "Ransomware",
        "trojan": "Trojan",
        "worm": "Worm",
        "spyware": "Spyware",
        "ddos": "DDoS",
        "distributed denial of service": "DDoS",
        "zero day": "Zero Days",
        "data breach": "Data Breach",
        "social engineering": "Social Engineering",
        "payments": "Payments",
        "breach": "Data Breach",
        "suspicius": "Suspicious Software",
        "password": "Password",
        "scam": "Scam",
    }
    lower_text = text.lower()
    categ = []
    for keyword, category in categories.items():
        if keyword in lower_text:
            categ.append(category)
    if not categ:
        categ.append("Other")
    return categ


# PARAMETROS DE TOKENIZACION Y ETIQUETADO
def prepare_data(df, tokenizer, max_length=512):
    # APLICAR ETIQUETAS LLAMANDO A CATEGORIZE
    df["label"] = df["text"].apply(categorize) 
    # TOKENIZAR
    encodings = tokenizer(
        df,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='tf' 
    )
    return encodings, df["label"].tolist()


# CARGAR MODELO PRE ENTRENADO Y TOKENIZADOR
def create_bert(model_name= 'bert-base-uncased'):
    model = TFBertModel.from_pretrained(model_name)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    return model, tokenizer


def split_data(df, labels):
    from sklearn.model_selection import train_test_split
    import tensorflow as tf
    x = df['text'] 
    y = labels
    return train_test_split(x,y,test_size = 0.2,random_state=35)


def tune_bert(df, labels):
    from sklearn.model_selection import train_test_split
    from transformers import TFBertForSequenceClassification
    x_train, x_test, y_train, y_test = split_data(df, labels)
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def tokenize_function(text):
        return tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='tf')
    
    train_encodings = tokenize_function(x_train)
    val_encodings = tokenize_function(x_test)

    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
    
    train_dataset = tf.data.Dataset.from_tensor_slices((dict(model(train_encodings)), y_train)).shuffle(100).batch(32)
    val_dataset = tf.data.Dataset.from_tensor_slices((dict(model(val_encodings)), y_test)).batch(16)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(train_dataset, validation_data=val_dataset, epochs=5)

    model.save_pretrained('tuned_bert_model')
    tokenizer.save_pretrained('tuned_bert_tokenizer')


# CREAR RED CONVOLUCIONADA
def create_cnn(bert_model, num_labels):
    input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name='attention_mask')
    bert_output = bert_model(input_ids, attention_mask=attention_mask)
    x = bert_output.last_hidden_state

    x = tf.keras.layers.Conv1D(filters=128,kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Conv1D(filers = 64, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)

    x = tf.keras.layers.GlobalMaxPooling1D()(x)

    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_labels, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    return model

    
#example of main
def main():
    # CARGAR DATA
    df = pd.read_csv('datasets/clean_data.csv')
    # CARGAR TOKENIZADOR Y MODELO PREENTRENADO
    model, tokenizer = create_bert(model_name='bert-base-uncased')
    # CANTIDAD DE ETIQUETAS QUE TIENE EL DF
    num_labels = 16 # this can change if you add or substract categories from categorize()
    # CREAR MODELO CONVOLUCIONADO
    cnn_model = create_cnn(model,num_labels)
    # EMPEZAR A TOKENIZAR UTILIZA TOKENIZADOR PRE ENTRENADO
    encodings,labels = prepare_data(df, tokenizer)
    # SE GUARDAN EN UN DATASET DE TENSORFLOW
    dataset = tf.data.Dataset.from_tensor_slices(({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask']    
    }, labels))
    
    dataset.batch(32)
    # COMPILAR MODELO    
    cnn_model.compile(optimizer= 'adam',loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])
    # INICIAR ENTRENAMIENTO DE MODELO
    cnn_model.fit(dataset, epochs=5)

# LLAMAR FUNCION MAIN
#if __name__ == "__main__":
#    main()
